# Load WAV file and perform CQT

In [274]:
// use std::fmt::Debug;

In [275]:
:dep hound = "3.5"
:dep cqt-rs = "0.1.0"
:dep ndarray = "0.15"
:dep image = "0.24"
:dep imageproc = "0.23"

In [276]:

use hound;
use cqt_rs::{CQTParams, Cqt};
use ndarray::Array2;
use image::{ImageBuffer, Rgb};
use imageproc::drawing::draw_filled_rect_mut;
use imageproc::rect::Rect;

In [277]:
use std::fs;


for entry in fs::read_dir(".").unwrap() {
    println!("{:?}", entry.unwrap().file_name())
}



".DS_Store"
"face.wav"
"rust_explore_constant_q_transform.ipynb"
"cmajor.wav"
"face.png"
"output.png"
".ipynb_checkpoints"


()

In [278]:


let mut reader = hound::WavReader::open("face.wav").unwrap();


In [279]:
println!("wav spec: {:?}, wav duration: {:?}", reader.spec(), reader.duration())

wav spec: WavSpec { channels: 1, sample_rate: 44100, bits_per_sample: 24, sample_format: Int }, wav duration: 352800


()

In [280]:
let sampling_rate:usize = reader.spec().sample_rate as usize;

println!("Sampling Rate: {}", sampling_rate);

Sampling Rate: 44100


In [281]:
let samples: Vec<i32> = reader.samples::<i32>().map(|s| s.unwrap()).collect();

In [282]:
println!("sample size: {}", samples.len());

sample size: 352800


In [283]:
let duration:f32 = samples.len() as f32 / sampling_rate as f32;

println!("sample duration in sec: {}", duration);

sample duration in sec: 8


### Convert i32 to f32

In [284]:
let input_signal: Vec<f32> = samples.iter().map(|&x| x as f32).collect();; // Your input audio signal

## Constant Q Transform via CQT

In [285]:
let cqt_params = CQTParams::new(
    100.0, // Min frequency
    20000.0_f32, // Max frequency
    25, // Number of bins
    sampling_rate, // Sampling rate
    sampling_rate/28 // Window length
).expect("Error creating CQTParams");

In [286]:
let cqt = Cqt::new(cqt_params);

In [287]:
let cqt_features:Array2<f32> = cqt.process(&input_signal, 1600).expect("Error computing CQT features");

In [288]:
&cqt_features.shape()

[220, 200]

##  Function to Convert CQT results into image

In [289]:
fn visualize_array2_as_image(input: &Array2<f32>, filename: &str) -> Result<(), Box<dyn std::error::Error>> {
    
    // Convert to DB value by taking log10 and times 20
    let array = input.mapv(f32::abs).mapv(f32::log10).mapv(|val| val * 20.0);
    
    // Normalize the values in the array to the [0, 255] range for visualization
    let min_val = array.iter().copied().reduce(f32::min).unwrap();
    let max_val = array.iter().copied().reduce(f32::max).unwrap();
    println!("min: {}, max: {}", min_val, max_val);
    
    let normalized_array = (255.0 * (array.to_owned().mapv(|x| (x - min_val) / (max_val - min_val)))).mapv(|x| x as u8);

    
    // Create an image buffer from the 2D array
    let width = normalized_array.shape()[0] as u32;
    let height = normalized_array.shape()[1] as u32;
    let mut img = ImageBuffer::new(width as u32, height as u32);

    // Fill the image buffer with pixel values
    for (x, y, pixel) in img.enumerate_pixels_mut() {
        let val = normalized_array[[x as usize, (height-1_u32-y) as usize]];
        *pixel = Rgb([val, 30, 30])
    }


    // Save the image to the specified filename
    img.save(filename)?;

    Ok(())
}

In [290]:
visualize_array2_as_image(&cqt_features, "output.png").unwrap();


min: 115.41423, max: 275.99924
